# Envoi d'une requête POST à l'API

In [1]:
import pandas as pd

# Lire les données du fichier CSV
file_path = '../data/processed/processed_data_test_sample.csv'
data = pd.read_csv(file_path)

# Supprimer les colonnes "Unnamed"
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Afficher les premières lignes du fichier CSV pour vérifier les colonnes
data.head()


,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,PREV_PRODUCT_COMBINATION_Cash X-Sell: low_MEAN,PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN,PREV_PRODUCT_COMBINATION_POS household with interest_MEAN,PREV_PRODUCT_COMBINATION_POS household without interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN,PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN,PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN,PREV_PRODUCT_COMBINATION_POS other with interest_MEAN,PREV_PRODUCT_COMBINATION_POS others without interest_MEAN
0,0,0,1,0,0,180000.0,1125000.0,78430.5,0.028663,-23782,...,0.0,0.0,0.00,0.5,0.00,0.0,0.0,0.0,0.0,0.0
1,0,1,0,1,0,112500.0,360000.0,28440.0,0.009334,-14193,...,0.0,0.0,1.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,90000.0,147217.5,6795.0,0.007120,-22021,...,0.1,0.2,0.00,0.0,0.00,0.0,0.0,0.0,0.1,0.0
3,0,1,0,1,0,76500.0,90000.0,4329.0,0.018850,-18155,...,0.0,0.0,0.25,0.0,0.25,0.0,0.5,0.0,0.0,0.0
4,0,0,0,0,0,81000.0,450000.0,19197.0,0.014520,-23579,...,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [2]:
import requests
import json

# URL de votre API déployée sur Azure
url = 'http://localhost:5000//predict'

# Préparer les en-têtes de la requête
headers = {
    'Content-Type': 'application/json'
}

# Fonction pour envoyer les requêtes POST et obtenir les prédictions
def get_predictions(input_data):
    response = requests.post(url, headers=headers, data=json.dumps({"input": input_data}))
    if response.status_code == 200:
        result = response.json()
        return result['predictions'], result['probabilities']
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None, None

# Boucle sur chaque ligne de données pour obtenir les prédictions
predictions = []
probabilities = []

for index, row in data.iterrows():
    input_data = row.tolist()
    prediction, probability = get_predictions(input_data)
    predictions.append(prediction)
    probabilities.append(probability)

# Ajouter les prédictions et probabilités au DataFrame, en gérant les None
data['predictions'] = [p[0] if p else None for p in predictions]
data['probabilities'] = [p[0] if p else None for p in probabilities]

# Afficher les résultats
data.head()

# Sauvegarder les résultats dans un nouveau fichier CSV
output_file_path = '../output/predictions_with_probabilities_local.csv'
data.to_csv(output_file_path, index=False)


In [3]:
# Afficher les résultats
data[['predictions', 'probabilities']].head()

,predictions,probabilities
0,0,"[0.5756586790084839, 0.4243413507938385]"
1,1,"[0.137717604637146, 0.862282395362854]"
2,1,"[0.1624525785446167, 0.8375474214553833]"
3,1,"[0.36546021699905396, 0.634539783000946]"
4,0,"[0.5947446823120117, 0.4052552878856659]"
